## Motivation

Goal-directed decision making is an essential aspect of our daily lives, as choices we make are shaped by the goals we want to achieve. An obvious difficulty one faces when trying to achieve a goal is how far ahead one should plan. In our everyday dynamic environment, the optimal planning depth is inevitably shaped by our goals and our knowledge about the underlying dynamics of the given environment. In other words, selecting a planning depth will depend on our preferences and our beliefs about the generative model of the task, namely causal relation between the actions, outcomes, state transitions, and the time remaining to fulfil the set goal. 

The choice of planning depth is naturally linked to the accuracy - complexity trade-off. The deeper the planning depth the more complex the plan is, however at the same time the deeper the planning depth the better one is in finding a behavioural policy most likely to fulfil the set goal. An implicit variable that shapes this trade-off also is the time available for planning. If one beliefs that the environment will go trough changes that would make our current plans obsolete, one does not need to plan past the moment when we expect the environment to change, however one has to act fast enough to reach the goal before the change occurs.

Here we will investigate individual variability in planning depth and elucidate mechanisms that cause this variability. We hypothesised that experience shapes planning depth in a way that modulates beliefs about required planning depth on a given task. These beliefs about planning depth interact with the beliefs about predictability of ones actions to form posterior beliefs about planning depth in a given task. 

## The space adventure task

<center><img  src="experiment.png"></center>

To probe mechanisms which influence inter-individual time evolution of the planning depth we have adapted a typical multi-stage task, which we will refer to as the Space Adventure Task (SAT). The SAT consists of participants navigate a spaceship through various planetary systems. (A) Each system consists of six planets and up to five planet types which cost or give fuel to the spaceship. The goal is to save as much fuel as possible until the end of the experiment consisting of $N=100$ mini-blocks. Units of fuel translate into one Euro cent, which is paid out to the participants as a performance dependent bonus.

(B) Throughout a mini-block, participants see the planets of the given planetary system, the current location of the spaceship, a coloured bar showing the remaining amount of fuel, and the number of remaining actions in the current mini-block (see subplot B). The configurations of planets and the starting position of the space ship are selected anew in each mini block in a way that participants have to plan forward to identify the best movement step. We instructed participants to travel from planet to planet and collect fuel for their spaceship, where each travel takes one action. Given the limited number of actions (either 2 or 3), the participants have to carefully choose a sequence of action which will result in the highest net return of fuel. In each trial within a mini-block, participants can make a choice between two actions: (i) They can travel into clock-wise direction to the next planet, or (ii) they perform a jump which leads them to a specific non-neighbouring planet. The jump transition pattern is shown in subplot B. Travelling to the next planet into clockwise direction costs 2 units and jumping costs 5 units of fuel.

Besides changing the maximum number of actions in each mini-block we have also varied the uncertainty of the jump action. In mini-blocks with low transition uncertainty, the spaceship would jump to the expected planet (as shown in Fig. 2C) with a $90\%$ probability. In mini-blocks with high transition uncertainty (clearly indicated to participants), the spaceship would jump to the expected planet only with $50\%$ probability. If the spaceship missed the expected planet it would land instead on one of the neighbouring planets with equal probability. Participants were told about the effect of uncertainty but were not told explicitly the probabilities. The goal is that they build a representation of success/fail probabilities over time.

In summary, there are four phases of the experiment, where we vary two factors 'total actions' (2 or 3) and 'uncertainty' (low and high). For each phase, we acquired choice and reaction time data from 25 mini-blocks, with 100 mini-blocks per participant in total. All go through an extensive training period to ensure that they have understood the task and memorised the effect of jumps for all six planet locations. 

## Computational model

To build an optimal model for planning (model that is completely informed about all aspects of the environment) we start by defining expectations over cumulative reward that an agent might collect starting from some state $s_t$ if she follows some policy $\pi$

$$ V[s_t|\pi, d] =  \sum_{\tau=t+1}^{t+d} \gamma^{\tau-t-1} E[r_\tau| s_t, \pi] \tag{1} $$

where $\pi$ denotes a policy (defined as state-action mapping, e.g.,$p_{\pi}(a|s)$), $\gamma$ some discount rate, and $E[ r_\tau| s_t, \pi]$ expected reward at some future time step $\tau$, and $d \in \{1, 2, 3\}$ the planning depth. We will consider that $\gamma=1$ for all mini-blocks as this parameter is not identifiable and represents another way to limit planning depth. 

The expected reward depends on agents knowledge of state transition probabilities $p(s'| s, a)$ and state-action reward mappings $p(r|s, a)$, that is

$$E[r_\tau| s_t, \pi] = \sum_{r_\tau} r_\tau p(r_\tau| s_t, \pi) \tag{2}$$

where

$$p(r_\tau|s_t, \pi) = \sum_{s_{t+1:\tau}, a_{t:\tau-1}} p(r_\tau|s_\tau, a_{\tau - 1}) \prod_{k=t+1}^\tau p(s_k|s_{k-1}, a_{k-1}) p(a_{k-1}|s_{k-1}) \tag{3}$$

The central issue here is to find the policy which maximises expected state value, that is, finding an optimal policy

$$\pi^*=\arg\max_\pi V[s_t|\pi, d] \tag{4}$$

One possible solution for this is dynamic programming, that is, a value iteration algorithm. This works well if the planning depth of an agent is clearly  limited, that is, a mini-block has relatively small number of trials.

If we define the state-action values at trial t as
$$
\begin{split}
Q(s_t, a_t, d) &= r_t(a_t,s_t) + \sum_{s^\prime} V[s^\prime|d-1]p(s_{t+1}=s^\prime|s_t, a_t) \\
V[s^\prime|d] &= \max_{a} Q(s^\prime, a, d) \\
r_t(a_t,s_t) &= C(a_t) + \sum_{s^\prime} r(s^\prime) p(s_{t+1}=s^\prime|s_t,a_t)
\end{split}
\tag{5}
$$


Here, $C(a_t)$ is the cost of executing action $a_t$, and $r_{t+1}$ is the reward of the destination planet. We can compute the $Q$-value at trial $t$ as a value iteration process starting from the state for which $V[s| d=0] = 0$, and computing iteratively Q-values for each state-action pair.

Given the computed Q values for each mini-block of the trial we can define the choice probability at current state and planning depth as 

$$p(a_t = 1|s_t, d) = \sigma(\beta * \Delta Q(s_t, d) + \theta) \tag{6}$$

where $\sigma(x) = \frac{1}{1+e^{-x}}$, $\Delta Q(s_t, d) = Q(a_t=1, s_t, d) - Q(a_t=0, s_t, d)$, $\beta$ denotes inverse response noise, and $\theta$ response bias (positive values imply bias toward choosing 'jump'), with $a_t=1$ corresponding to 'jump' choice, $a_t=0$ to 'right' response.

Finally, we will assume that the state transition probabilities $p(s'| s, a)$ are learned over time and adjusted based on the experienced sequence of successful and failed jumps. If we define with 

$$p(s_{t+1} = target|s_t, a_t=1)=\rho_t, \tag{7}$$

the probabilty that the jump will end in a target state, we can define the following update equations for $\rho$ depending on the experienced success ($o_t=1$) or failure ($o_t = 0$) of a jump

$$ \rho_t = \rho_{t-1} + \alpha (o_t - \rho_{t-1}) \tag{8}$$

Note that the learning rate parameter $\alpha$ influences how fast subjects change their beliefs about the probability that a jump will be 'successful'. The larger the $\alpha$ the faster they forget past experience and rely only on recent outcomes. 

## Planning depth and parameter inference

The parametric generative model is expressed in the unconstrained space of parameters, meaning that each parameter is mapped to a real axis, depending on its definition. As the learning rate $\alpha$ is constrained on a [0, 1] interval, the unconstrained parameter $\tilde{\alpha}$ is obtained from a logit transform of $\alpha$, that is
$\tilde{\alpha} = logit(\alpha)$ and $\alpha = logistic(\tilde{\alpha})$. Similarly, we use $\tilde{\beta} = \ln(\beta)$ and $\beta = e^{\tilde{\beta}}$ to define the transform of the inverse decision noise $\beta$. As response bias $\theta$ is unconstrained already we have $\theta = \tilde{\theta}$. 

For inference of the free model parameter $\gamma = (\tilde{\alpha}, \tilde{\beta}, \tilde{\theta})$ (which should not be confused with the discounting parameter $\gamma$ in equation 5 above) and the trial-dependent planning depth $d_t$ we use the following hierarchial generative model

$$ p(G)\prod_{n=1}^N p(\gamma_n|G) p(r_n|\alpha_n) \prod_{b=1}^{100} \prod_{t=1}^{t_{max}(b)} p(a^n_{t,b}|s^n_{t,b}, r_n(b), \gamma_n), \tag{9}$$

where $n$ denotes the participant's number, $b$ the mini-block, and $G$ the group-level hyper-priors (see below), and $r_n(b)$ denotes a participant-specific prior over the planning depth, dependent on the current phase of the experiment. In other words, $p(d^n_{b, t}=d|r_n) = r_{n,d}$, $\sum_d r_{n,d} = 1$. Note that given a posterior distribution over model parameters

$$ P(G, \gamma_{1:N}, r_{1:N}| A_n, S_n, \alpha_n) \approx Q(G, \gamma_{1:N}, r_{1:N}), \tag{10}$$

where $A_n$, $S_n$ denote the set of all responses and states (stimuli) of the $n$th participant, we can obtain the marginal posterior over a planning depth at any trial and mini-block as 

$$ Q(d_{t, b}^n) = \frac{1}{K} \sum_{i=1}^K \frac{p(a^n_{t,b}|s^n_{t,b}, d^n_{t, b}, \gamma_{n, i}) p(d^n_{t, b}|r_{n,i})}{p(a^n_{t,b}|s^n_{t,b}, r_{n,i}, \gamma_{n,i})} \tag{11}$$

where $i$ denotes posterior samples from the marginal posterior distribution $Q(\gamma_n, r_n)$ of the $n$th participant.

### Hierarchical prior distribution

The group-level hyper-prior over $\gamma_n$ is defined as a normal-inverse-gamma distribution for each free parameter:

$$ p(G) = p(g_\tilde{{\alpha}}, g_\tilde{{\beta}}, g_\tilde{{\theta}}, \sigma_\tilde{{\alpha}}^2, \sigma_\tilde{{\beta}}^2, \sigma_\tilde{{\theta}}^2) = \prod_{i \in \{\tilde{\alpha}, \tilde{\beta}, \tilde{\theta}\}}\mathcal{N}-\Gamma^{-1}\left(g_i, \sigma_i^2; m_i, \lambda_i, w_i, b_i\right), \tag{12}$$

where $g_{\tilde{\alpha}}$ and $\sigma_{\tilde{\alpha}}^2$ are the mean and variance of the logit-transformed learning rate $\alpha$, and depend on the four parameters of the normal-inverse-gamma distribution $m_i, \lambda_i, w_i, b_i$. We use the normal-inverse-gamma distribution as it is the conjugate prior to a multivariate normal distribution with unknown variance. Then the subject-level prior is defined as a (multivariate) normal distribution

$$p(\gamma_n|G) = p(\tilde{\alpha_n}, \tilde{\beta_n}, \tilde{\theta_n} | G) =\prod_{i \in \{\tilde{\alpha}, \tilde{\beta}, \tilde{\theta}\}} \mathcal{N}\left(\gamma_{n,i}; g_i, \sigma_i^2 \right) \tag{13}$$

Finally, the prior over planning depth corresponds to a Dirichlet distribution, where 

$$ p(r_n|\alpha_n) = \prod_{h = 1}^4 Dir(\alpha_{n,h}) \tag{14}$$

where $h$ denotes the four different phases of the experiment, hence depending on the value of the mini-block the function
$r_n(b)$ selects the appropriate prior value $r_n^h$. Note that $\alpha_{n,h}$ is the argument of the dirichlet distribution and is not to be confused with the learning rate $\alpha$.

### Variational approximation

The approximate posterior $Q(G, \gamma_{1:N}, r_{1:N})$ from equation 10 is factorised as follows:

$$Q(G, \gamma_{1:n}, r_{1:n}) = Q(G)\prod_{n=1}^NQ(\gamma_n)Q(r_n), \tag{15}$$

where 

$$ Q(G) = Q(g_\tilde{{\alpha}}, g_\tilde{{\beta}}, g_\tilde{{\theta}}, \sigma_\tilde{{\alpha}}^2, \sigma_\tilde{{\beta}}^2, \sigma_\tilde{{\theta}}^2) = \frac{1}{8\sigma^2_\tilde{\alpha} \sigma^2_\tilde{\beta} \sigma^2_\tilde{\theta}}\mathcal{N}_6\left(\mu, \Sigma\right) \tag{16}$$

$$ Q(\gamma_n) = \mathcal{N_3}\left(m_n, \Upsilon_n \right) \tag{17}$$

and

$$ Q(r_n) = \prod_{h = 1}^4 Dir(\alpha^{\prime}_{n, h}) \tag{18}.$$

Note that the definition of the posterior over $G$ (equation $16$) is not simply a six-dimensional Gaussian distribution, as the variances can only be positive. For finding the parameters of the approximate posterior, and additional free model parameters $(m_i, \lambda_i, w_i, b_i)$ we use the stochastic vartiational inference scheme from Pyro. More details on this can be found here:

http://pyro.ai/examples/svi_part_i.html

https://pyro.ai/examples/svi_part_iii.html

### Planning depth posterior

Given a known set of model parameters $\gamma_{n}$ and priors over the planning depth $r_n$ we obtain the posterior over planning depth (at trial $t$ of a mini-block $b$) as

$$ Q(d_{t, b}^n| \gamma_n, r_n) = \frac{p(a^n_{t,b}|s^n_{t,b}, d^n_{t, b}, \gamma_{n}) p(d^n_{t, b}|r_{n})}{p(a^n_{t,b}|s^n_{t,b}, r_{n}, \gamma_{n})}. \tag{19}$$

Hence for a given approximate marginal posterior distribution $Q(\gamma_n, r_n)$ we can obtain the marginal (approximate) posterior over planning depths as  

$$ Q(d_{t, b}^n) = \frac{1}{K} \sum_{i=1}^K \frac{p(a^n_{t,b}|s^n_{t,b}, d^n_{t, b}, \gamma_{n, i}) p(d^n_{t, b}|r_{n,i})}{p(a^n_{t,b}|s^n_{t,b}, r_{n,i}, \gamma_{n,i})} \tag{20}$$

where $i$ denotes posterior samples from the marginal posterior distribution $Q(\gamma_n, r_n)$ of the $n$th participant.

### Exceedance probability of planning depth

The exceedance probability quantifies how likely it is that a specific planning depth $d$ is the most likely a posteriori. Hence for fixed $\gamma_n$ and $r_n$ the most likely planning depth a posteriori corresponds to the mode of the posterior hence

$$d^*|\gamma_n, r_n = {\arg \max}_d Q(d_{t,b}^n = d|\gamma_n, r_n) \tag{21}$$

Hence the planning depth exceedance probability at trial $t$ and mini-block $b$ is obtained as an average a sample from the approximate posterior 

$$EP(d_{t,b}^n) = \frac{1}{K} \sum_i \delta_{d^*| \gamma_{n,i}, r_{n,i}, d_{t, b}^n}, \tag{22}$$ 

where $\delta_{i,j} = 1$ for $i=j$ and $\delta_{i,j} = 0$, for $i \neq j$.

## Bayesian linear regression

Here I am using a QR decomposition (https://mc-stan.org/users/documentation/case-studies/qr_regression.html) combined with a hierarchical model to estimate per participant regression parameters. The posterior is estimated using NUTS sampling. 

For more info see the notebooks inside the folder https://github.com/dimarkov/pybefit/tree/master/examples/sparse_regression in the examples folder.